In [1]:
!pip install --pre deepchem

     |████████████████████████████████| 608 kB 7.8 MB/s 
     |████████████████████████████████| 20.6 MB 1.7 MB/s 


In [2]:
import deepchem as dc

In [30]:
dataset_file = 'molnet_bace.csv'
tasks = ['Class']
featurizer_func = dc.feat.ConvMolFeaturizer()

loader = dc.data.CSVLoader(tasks=task, feature_field='mol', id_field='CID', featurizer=featurizer_func)
dataset = loader.create_dataset(dataset_file)

transformer = dc.trans.BalancingTransformer(dataset=dataset)
dataset = transformer.transform(dataset)

splitter = dc.splits.RandomSplitter()
train_dataset, test_dataset = splitter.train_test_split(dataset)

In [31]:
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification')
model.fit(train_dataset, nb_epoch=50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_14:0", shape=(628,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_13:0", shape=(628, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_17:0", shape=(3318,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/

0.13367197036743164

In [61]:
pred_train_y_prob = model.predict(train_dataset).reshape(-1, 2)
train_pred_list = []
for i, train_prob in enumerate(pred_train_y_prob):
    pred_pos_prob = train_prob[1]
    train_pred_list.append(pred_pos_prob)
train_y = train_dataset.y.reshape(-1)

pred_test_y_prob = model.predict(test_dataset).reshape(-1, 2)
test_pred_list = []
for i, test_prob in enumerate(pred_test_y_prob):
    pred_pos_prob = test_prob[1]
    test_pred_list.append(pred_pos_prob)
test_y = test_dataset.y.reshape(-1)

In [62]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import numpy as np

gnn_train_auc = round(roc_auc_score(train_y, train_pred_list),2)
gnn_train_acc = round(accuracy_score(train_y, np.round(train_pred_list)),2)
gnn_train_sen = round(recall_score(train_y, np.round(train_pred_list)),2)
gnn_train_spc = round((gnn_train_acc * len(train_y)- gnn_train_sen * train_y.sum())/(len(train_y)-train_y.sum()),2)

gnn_test_auc = round(roc_auc_score(test_y, test_pred_list),2)
gnn_test_acc = round(accuracy_score(test_y, np.round(test_pred_list)),2)
gnn_test_sen = round(recall_score(test_y, np.round(test_pred_list)),2)
gnn_test_spc = round((gnn_test_acc * len(test_y)- gnn_test_sen * test_y.sum())/(len(test_y)-test_y.sum()),2)

In [63]:
gnn_perf = {'AUC':[gnn_train_auc, gnn_test_auc],
            'ACC':[gnn_train_acc, gnn_test_acc],
            'SEN':[gnn_train_sen, gnn_test_sen],
            'SPC':[gnn_train_spc, gnn_test_spc]}

In [64]:
gnn_perf

{'ACC': [0.96, 0.83],
 'AUC': [1.0, 0.89],
 'SEN': [0.99, 0.86],
 'SPC': [0.94, 0.8]}

# scaffold splitter

In [74]:
# scaffold splitter
dataset_file = 'molnet_bace.csv'
tasks = ['Class']
featurizer_func = dc.feat.ConvMolFeaturizer()

loader = dc.data.CSVLoader(tasks=task, feature_field='mol', featurizer=featurizer_func)
dataset = loader.create_dataset(dataset_file)

transformer = dc.trans.BalancingTransformer(dataset=dataset)
dataset = transformer.transform(dataset)

scaffoldsplitter = dc.splits.ScaffoldSplitter()
train,test = scaffoldsplitter.train_test_split(dataset)
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification')
model.fit(train_dataset, nb_epoch=50)

pred_train_y_prob = model.predict(train_dataset).reshape(-1, 2)
train_pred_list = []
for i, train_prob in enumerate(pred_train_y_prob):
    pred_pos_prob = train_prob[1]
    train_pred_list.append(pred_pos_prob)
train_y = train_dataset.y.reshape(-1)

pred_test_y_prob = model.predict(test_dataset).reshape(-1, 2)
test_pred_list = []
for i, test_prob in enumerate(pred_test_y_prob):
    pred_pos_prob = test_prob[1]
    test_pred_list.append(pred_pos_prob)
test_y = test_dataset.y.reshape(-1)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import numpy as np

gnn_train_auc = round(roc_auc_score(train_y, train_pred_list),2)
gnn_train_acc = round(accuracy_score(train_y, np.round(train_pred_list)),2)
gnn_train_sen = round(recall_score(train_y, np.round(train_pred_list)),2)
gnn_train_spc = round((gnn_train_acc * len(train_y)- gnn_train_sen * train_y.sum())/(len(train_y)-train_y.sum()),2)

gnn_test_auc = round(roc_auc_score(test_y, test_pred_list),2)
gnn_test_acc = round(accuracy_score(test_y, np.round(test_pred_list)),2)
gnn_test_sen = round(recall_score(test_y, np.round(test_pred_list)),2)
gnn_test_spc = round((gnn_test_acc * len(test_y)- gnn_test_sen * test_y.sum())/(len(test_y)-test_y.sum()),2)

gnn_perf = {'AUC':[gnn_train_auc, gnn_test_auc],
            'ACC':[gnn_train_acc, gnn_test_acc],
            'SEN':[gnn_train_sen, gnn_test_sen],
            'SPC':[gnn_train_spc, gnn_test_spc]}

gnn_perf

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/Reshape_14:0", shape=(704,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/Reshape_13:0", shape=(704, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/Reshape_17:0", shape=(3342,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_7/

{'ACC': [0.97, 0.78],
 'AUC': [1.0, 0.86],
 'SEN': [0.96, 0.76],
 'SPC': [0.98, 0.8]}